# Análisis de regresión: variables dicotómicas

Una variable dicotómica toma solo dos valores, generalmente 0 y 1, y se utiliza para representar categorías. Por ejemplo:
- Genero_femenino = 1 si el individuo es mujer, 0 si es hombre.
- Sucursal_Norte = 1 si pertenece a la sucursal del norte, 0 en caso contrario.

En un modelo de regresión lineal, el coeficiente β de una variable dicotómica se interpreta como la diferencia en el valor promedio de la variable dependiente entre el grupo codificado como 1 y el grupo de referencia (codificado como 0), manteniendo constantes las demás variables.

*Interpretación de coeficientes de regresión para variables dicotómicas*

Supón que estimas el siguiente modelo:

Ventas = β₀ + β₁ * Publicidad + β₂ * Genero_femenino + ε

Y el resultado de la regresión es:
- β₀ = 120
- β₁ = 5.2
- β₂ = 18.7

La interpretación es:
- Por cada unidad adicional de gasto en publicidad, las ventas aumentan en promedio 5.2 unidades (β₁).
- Si el cliente es mujer (Genero_femenino = 1), las ventas aumentan en promedio 18.7 unidades en comparación con hombres (Genero_femenino = 0), manteniendo constante el gasto en publicidad (β₂).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

## 9.1. Variables dicotómicas

El archivo *ubermty* contiene información de viajes en taxi o uber en la ciudad de Monterrey de junio de 2016 a agosto de 2017. Crearemos un modelo para estimar la duración de cada viaje en función de la distancia y el día de la semana.

In [ ]:
variables=['id', 'dropoff_longitude', 'dropoff_latitude', 
           'Dia', 'DuracionMinutos', 'DistanciaKm']

# Cargar los datos
df = pd.read_excel('https://github.com/adan-rs/AnalisisDatos/raw/main/data/ubermty.xlsx', usecols=variables)

In [ ]:
# Revisar las variables
df.info()

In [ ]:
# Obtener la estadística descriptiva
df.describe().T

In [ ]:
# Filtrado de valores atípicos
Q1 = df['DuracionMinutos'].quantile(0.25)
Q3 = df['DuracionMinutos'].quantile(0.75)
IQR = Q3-Q1
lim_inf = Q1-1.5*IQR
lim_sup = Q3+1.5*IQR
df = df[(df['DuracionMinutos']>lim_inf)&(df['DuracionMinutos']<lim_sup)]

In [ ]:
# Generar un gráfico
sns.scatterplot(x="dropoff_longitude",
                y="dropoff_latitude",
                data=df)

In [ ]:

import folium
# Calcula centro del mapa con la media de las coordenadas
map_center = [df['dropoff_latitude'].mean(), df['dropoff_longitude'].mean()]
# Crea un mapa alrededor del centro con un nivel inicial de zoom
mapa = folium.Map(location=map_center, zoom_start=10)
# Recorre cada fila, crea un marcador circular y lo añade al mapa
for index, row in df.iterrows(): 
    folium.CircleMarker(
        location=[row['dropoff_latitude'], row['dropoff_longitude']],
        radius=3, color='blue', fill=True, fill_color='blue', 
        fill_opacity=0.6 
    ).add_to(mapa)
mapa

In [ ]:
sns.boxplot(x='Dia', y='DuracionMinutos', data=df);

In [ ]:
df.head()

**Creación de variables dicotómicas**: La función `get_dummies` convierte una variable categórica (con k categorías) en k variables dicotómicas. Entre otros parámetros se puede establecer:
- prefix: agrega un prefijo a los nombres de las columnas.
- dtype: de manera predeterminada está en *booleano*, se puede cambiar a *int*
- drop_first: quita la columna con la primera categoría

Puedes usar esta función para crear un nuevo dataframe de las variables dummy o puedes aplicarlo directamente al dataframe.

Es importante considerar que
- Si una variable categórica tiene más de dos categorías, se deben crear k – 1 variables dummy para evitar multicolinealidad (esto se puede automatizar con drop_first=True en pd.get_dummies()).
- El grupo que se omite (referencia) es el que se considera como base para las comparaciones.
- Las variables dummy permiten capturar efectos de pertenencia a una categoría específica, no cambios graduales como ocurre con variables continuas.


In [ ]:
# Crear variables dicotómicas
df = pd.get_dummies(df, columns=['Dia'], prefix='dia',  
                    dtype=int, drop_first=True)
df.head(3)

In [ ]:
df.columns

In [ ]:
X = df[['DistanciaKm', 'dia_lunes', 'dia_martes', 'dia_miercoles', 
        'dia_jueves', 'dia_viernes', 'dia_sabado']]  
# Añadir una columna de unos para el intercepto
X = sm.add_constant(X)
y = df['DuracionMinutos']

In [ ]:
# Ajustar el modelo de regresión
model = sm.OLS(y, X).fit()

# Imprimir los resultados
print(model.summary())

In [ ]:
pronostico = model.predict([[1, 10, 0, 0, 0, 0, 1, 0]])
pronostico